In [ ]:
%matplotlib inline
# from __future__ import print_function
# from __future__ import division

import os
from tifffile import imread, imsave
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import cv2
#import sima
#import sima.motion
#import sima.segment
import scipy
import glob
import os
from PIL import Image
from skimage import io
import h5py
import time
import scipy.io
import shutil
import ntpath

In [ ]:
# set parameters
magnitude = 105/50
sensor_size = np.asarray([11.264,5.984])*1000 #sensor size of camera, in um
image_max_size = sensor_size/magnitude # in um
pixel_max_size = np.asarray([2040,1086])# in pixel 
size_per_pixel = image_max_size/pixel_max_size# in um/pixel
size_per_pixel_single = np.mean(size_per_pixel)# in um/pixel
size_per_pixel_down_sample = 10 # in um, the desired spational resolution for down sampling
reso_pixel_down_sample = int(np.ceil(size_per_pixel_down_sample/size_per_pixel_single))

In [ ]:
# path_batch = [
#                 'L:/Wild_type/AQ1/20190520/md_1/processed/all_image_down_sampled_1_rig.tif',  
#                 'L:/Wild_type/AQ3/20190523/md_1/processed/all_image_down_sampled_1_rig.tif',            
#                 'L:/Wild_type/AQ4/20190524/md_1/processed/all_image_down_sampled_1_rig.tif',
#                 ]
# path_save = 'C:/Users/yatang/Google Drive/optic imaging/'
# file_save = ['AQ1','AQ3','AQ4']
# for i,path in enumerate(path_batch):
#     image = imread(path)
#     image_save = image[:200,:,:]
#     imsave(path_save+file_save[i]+'.tif',image_save,imagej=True)


In [ ]:
# image = imread(path)

In [ ]:
# ntpath.abspath(path)

In [ ]:
def file2int(x):
    filename = os.path.splitext(x)[0]
    idx_last = [m.start() for m in re.finditer('_', filename)][-1]
    return int(filename[idx_last+1:])

In [ ]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [ ]:
# get started
path_0_batch = [
#                 'L:/Wild_type/AO2/20181031/',  
#                 'L:/Wild_type/AO1/20181101/',            
#                 'L:/Wild_type/AO1/20181105/',
#                 'L:/Wild_type/AO3/20181102/',
#                 'L:/Wild_type/AO3/20181106/'
#                 'L:/Wild_type/AO1/20181207/'
#                 'L:/Wild_type/AP1/20181210/'
#                 'L:/Wild_type/AP1/20181212/'
#                 'L:/Wild_type/AP3/20181214_left_eye/',
#                 'L:/Wild_type/AP3/20181214_right_eye/'
#                 'L:/Wild_type/AO1/20190122/',
#                 'L:/Wild_type/AO1/20190204/',
#                 'L:/Wild_type/AJ1/20190116/',
#                 'L:/Wild_type/AJ2/20190111/',
#                 'L:/Wild_type/AJ3/20190116/',
#                 'L:/Wild_type/AK1/20190116/',
#                 'L:/Wild_type/AK2/20190116/',
#                 'L:/Wild_type/AQ1/20190111/',
#                 'L:/Wild_type/AQ1/20190207/',
#                 'L:/Wild_type/AQ2/20190116/',
#                 'L:/Wild_type/AQ2/20190208/',
#                 'L:/Wild_type/AP1/20190214/',
#                 'L:/Wild_type/AP3/20190215/',
#                 'L:/Wild_type/AQ3/20190211/',
#                 'L:/Wild_type/AQ4/20190212/',
#                 'L:/Wild_type/AO1/20190122/',
    
#                 'L:/Wild_type/AL1/20190412/',
#                 'L:/Wild_type/AO1/20190417/',
#                 'L:/Wild_type/AP1/20190418/',
#                 'L:/Wild_type/AP3/20190418/',
#                 'L:/Wild_type/AM1/20190419/',
#                 'L:/Wild_type/AQ2/20190419/',
#                 'L:/Wild_type/AR1/20190423/',
#                 'L:/Wild_type/AQ1/20190520/',
#                 'L:/Wild_type/AQ3/20190523/',
#                 'L:/Wild_type/AQ4/20190524/',
    
    
    'L:/Wild_type/AQ1/20190829/',
    'L:/Wild_type/AQ3/20190829/',
    'L:/Wild_type/AQ3/20190829_2/',
    'L:/Wild_type/AQ4/20190830/',

    
               ]

path_1 = [
#             'md_0/',
#             'mb_0/',
#             'fg_0/',
#             'md_1/',
#             'mb_1/',
#             'fg_1/',
#             'md_2/',
#             'mb_2/',
#             'fg_2/',
#             'loom_tile/',
#             'optic_flow/'，
            'retino_reso/'
           ]


In [ ]:
# down sample the images
for path_0 in path_0_batch:
    print(path_0_batch)
#     print(path_0) 
    path_all = []
    for path in path_1:
        if os.path.exists(path_0+path):
            path_all.append(path_0+path)

    for path in path_all:
        if not os.path.exists(path+'raw/'):
            os.mkdir(path+'raw/')

    for path in path_all:
        files = os.listdir(path)
        for f in files:
            if os.path.isfile(path+f):
                shutil.move(path+f, path+'raw/'+f)

    for path in path_all:
        tiffiles_raw = [os.path.join(root, name)
                     for root, dirs, files in os.walk(path+'raw/')
                     for name in files
                     if name.endswith((".tif", ".tiff"))]
        tiffiles = sorted(tiffiles_raw,key = file2int)
        num_image = len(tiffiles)
        image = imread(tiffiles[0])
        blur = cv2.blur(image,(reso_pixel_down_sample*2,reso_pixel_down_sample*2))
        image_down_sampled = blur[::reso_pixel_down_sample,::reso_pixel_down_sample]
        [row,col]=np.shape(image_down_sampled)
    #     row_del_start = 20
    #     row_del_end = -20
    #     col_del_start = 30
    #     col_del_end = -30
    #     image_all = np.empty([num_image,row-np.int(np.abs(row_del_start)+np.abs(row_del_end)),col-np.int(np.abs(col_del_start)+np.abs(col_del_end))], dtype='uint16')
        image_all = np.empty([num_image,row,col], dtype='uint16')
        # input and down-sample images
        print('loading and down-sampling images')
        i=0
        for tiff in log_progress(tiffiles):
            image = imread(tiff)
            image_blur = cv2.blur(image,(reso_pixel_down_sample*2,reso_pixel_down_sample*2))
            image_down_sampled = image_blur[::reso_pixel_down_sample,::reso_pixel_down_sample]
    #         image_all[i,:,:] = image_down_sampled[row_del_start:row_del_end,col_del_start:col_del_end]
            image_all[i,:,:] = image_down_sampled
        #    print(i,'of',num_image)
            i+=1

        path_save = path+'processed/'
        if not os.path.exists(os.path.dirname(path_save)):
            os.makedirs(path_save)
        if ('loom' in path) or ('md' in path) or ('optic_flow' in path):
            [num_image,_,_] = image_all.shape
            num_half = long(num_image/2)
            image_1 = image_all[:num_half,:,:]
            imsave(path_save+'all_image_down_sampled_1.tif',image_1,imagej=True)
            image_2 = image_all[num_half:,:,:]
            imsave(path_save+'all_image_down_sampled_2.tif',image_2,imagej=True)
        else:
            imsave(path_save+'all_image_down_sampled.tif',image_all,imagej=True)
        print('done:'+path_save)

In [ ]:
tiff

In [ ]:
# rotate or cut images if necessary
# important: only run once!!!
for path_0 in path_0_batch:
    print(path_0_batch)
    path_all = []
    for path in path_1:
        if os.path.exists(path_0+path):
            path_all.append(path_0+path)
       
    flag = True    
    for path in log_progress(path_all):
        tiffiles = [os.path.join(root, name)
                     for root, dirs, files in os.walk(path+'processed/')
                     for name in files
                     if name.endswith((".tif", ".tiff"))]
        for i,tif in enumerate(tiffiles):
            image = imread(tif)
            [n_frame, n_row, n_col] = np.shape(image)
#             image_temp = np.rot90(image[0,:155,:300])
#             image_temp = np.flip(image[0,:,:],1)
            image_temp = np.rot90(image[0,:,:],2)
            [n_row_new, n_col_new] = np.shape(image_temp)
            image_new = np.empty([n_frame,n_row_new, n_col_new], dtype='uint16')
            for j in range(n_frame):
#                 image_new[j,:,:] = np.rot90(image[j,:155,:300])
#                 image_new[j,:,:] = np.flip(image[j,:,:],1)
#                 image_new[j,:,:] = np.rot90(np.flip(image[j,:,:],1),2)
                image_new[j,:,:] = np.rot90(image[j,:,:],2)
            imsave(tif,image_new,imagej=True)
            
            

In [ ]:
#run motion correction using MATLAB
#load motion-corrected images 
# save roi-based files to .flr file to be read by MATALB

for path_0 in path_0_batch:
    print(path_0_batch)
    path_all = []
    for path in path_1:
        if os.path.exists(path_0+path):
            path_all.append(path_0+path)
       
    flag = True    
    for path in log_progress(path_all):
        tiffiles = [os.path.join(root, name)
                     for root, dirs, files in os.walk(path+'processed/')
                     for name in files
                     if name.endswith(("rig.tif", "rig.tiff"))]
        
        if np.size(tiffiles)==0:
            continue
            
        for i,tif in enumerate(tiffiles):
            image_mc = imread(tif)
            if i==0:
                image_all_mc = image_mc
            else:
                image_all_mc = np.concatenate((image_all_mc, image_mc), axis=0)

        [n_frame, n_row, n_col] = np.shape(image_all_mc)
        datapara = np.zeros(100)
        datapara[0] = int(n_row*n_col) #'numofrois'        
        datapara[1] = 10;#'samplingrate' unit Hz
        datapara[2] = 1/datapara[1];#'frametime'unit sec
        datapara[3] = n_col;#'Xres'
        datapara[4] = n_row;#'Yres'
        datapara[5] = datapara[3]*size_per_pixel_down_sample #'Xscale' 
        datapara[6] = datapara[4]*size_per_pixel_down_sample #'Yscale'
        if flag:
            x_reso = n_col
            y_reso = n_row
            x_scale = datapara[5]
            y_scale = datapara[6]
            scipy.io.savemat(path_0+'image_info.mat',mdict={'x_reso': x_reso,'y_reso':y_reso,
                                                            'x_scale':x_scale,'y_scale':y_scale})
            flag = False

        datapara[7] = 4 #'procedure'
        datapara[8] = 12 #'numofstim'
        if path[-5:-3:] == 'fg':
            datapara[9] = 5 #'repetition'
        else:
            datapara[9] = 10 #'repetition'
        datapara[10]= n_frame #'numofframes'
        datapara[11] = int(n_frame/(datapara[8]*datapara[9])) #'frames_per_trig'
        trig = np.zeros(n_frame)
        if path[-14:-3] == '20180924/mb':
            temp = np.where(np.mod(range(470*12*10),6)==0)
            trig_idx = [0]
            for i in range(1,9400-2):
                if np.mod(temp[0][i],470)>np.mod(temp[0][i+1],470):
                    trig_idx.append(i)
            trig[trig_idx] = 1
        else:
            trig[::int(datapara[11])]=1
        print('the number of trig is:'+str(np.size(np.where(trig==1))))    
        path_h5 = path+'processed/'+path[path.index('201')::].replace('/','_')[:-1:]+'.h5'
        hf = h5py.File(path_h5, 'w')
        hf.create_dataset('datapara', data=datapara)
        hf.create_dataset('data', data=image_all_mc)
        hf.create_dataset('trig', data=trig)
        hf.close()
    
    

In [ ]:
image_all_mc[1,:10,:10]

In [ ]:
image_all_mc[0,:10,:10]

In [ ]:
# image[1,:,:] = np.rot90(np.flip(image[1,:,:],1),2)
# image_new[j,:,:] = np.rot90(image[j,:,:],2)
plt.imshow(image,cmap='gray',aspect='equal')   
plt.show()
plt.imshow(np.rot90(image,2),cmap='gray',aspect='equal')   
plt.show()

In [ ]:
path_0_batch = [
                'L:/Wild_type/AO1/20181101/'            
                'L:/Wild_type/AO1/20181105/',
                'L:/Wild_type/AO3/20181102/',
                'L:/Wild_type/AO3/20181106/'
               ]
for path_0 in path_0_batch:
    path_1 = [
                'md_0/',
                'md_1/',
                'md_2/',
                'mb_0/',
                'mb_1/',
                'mb_2/',
                'fg_0/',
                'fg_1/',
                'fg_2/',
                'loom_tile/',
               ]

    path_all = []
    for path in path_1:
        path_all.append(path_0+path)
        
    flag = True    
    for path in log_progress(path_all):
        tiffiles = [os.path.join(root, name)
                     for root, dirs, files in os.walk(path+'processed/')
                     for name in files
                     if name.endswith(("rig.tif", "rig.tiff"))]

In [ ]:
x_reso = n_col
y_reso = n_row
x_scale = datapara[5]
y_scale = datapara[6]
scipy.io.savemat(path_0+'image_info.mat',mdict={'x_reso': x_reso,'y_reso':y_reso,
                                                'x_scale':x_scale,'y_scale':y_scale})

In [ ]:
# run motion correction
print('running motion correction')

data_temp=np.expand_dims(np.expand_dims(image_all,axis=1),axis=4)
sequences = [sima.Sequence.create('ndarray', data_temp)]
dataset_path = path+'processed/all_image_PlaneTranslation2D.sima'
correction_approach_PlaneTranslation2D = sima.motion.PlaneTranslation2D(max_displacement = [30, 30])   
dataset = correction_approach_PlaneTranslation2D.correct(sequences, dataset_path, trim_criterion=0.95)
print("done motion correction. Saving...")
# Generate the output filenames with Python list comprehensions.
size_limit = 4e9
if image_all.nbytes>size_limit:
#    output_filenames = [[[images_all_file.replace('.tif', '_corrected_PlaneTranslation2D.tif')]]]
    output_filenames_hdf5 = [[[path+'processed/all_image_corrected_PlaneTranslation2D.h5']]]
    # The resulting filenames are printed for clarification.
    print("Output filenames:\n")
    print(output_filenames_hdf5)

    # Export the corrected frames for a presentation.
#    dataset.export_frames(output_filenames, fill_gaps=True)
    dataset.export_frames(output_filenames_hdf5, fmt='HDF5', fill_gaps=True)
    print("\nDone exporting!")


else:
    output_filenames =  [[[path+'processed/all_image_corrected_PlaneTranslation2D.tif']]]
#        output_filenames_hdf5 = [images_all_file.replace('.tif', '_corrected_PlaneTranslation2D.h5')]
    # The resulting filenames are printed for clarification.
    print("Output filenames:\n")
    print(output_filenames)

    # Export the corrected frames for a presentation.
    dataset.export_frames(output_filenames, fill_gaps=True)
#        dataset.export_frames(output_filenames_hdf5, fmt='HDF5', fill_gaps=True)
    print("\nDone exporting!")             